### Objective:
To classify images in the Caltech-256 dataset, which is an improvement over Caltech-101 dataset using a Convolutional Neural Network.

### Problem Statement
To build and implement a Convolutional Neural Network model to classify images in the Caltech-256 dataset.

At the end of this competition, you will be able to:

* Load and extract features of images available in the Caltech-256 dataset using ImageDataGenerator

* Build convolutional neural networks using either Keras or PyTorch deep learning libraries

* Use the pre-trained models using either Keras or PyTorch deep learning libraries

### Description:
Caltech-256 is an object recognition dataset containing approximately 30,000 real-world images, of different sizes, spanning 256 classes (256 object classes and an additional clutter class). Each class is represented by at least 80 images. The dataset is a superset of the Caltech-101 dataset.

Here is a handy link to Kaggle's competition documentation (https://www.kaggle.com/docs/competitions), which includes, among other things, instructions on submitting predictions (https://www.kaggle.com/docs/competitions#making-a-submission).

### Instructions for downloading train and test data are as follows:

### 1. Create an API key in Kaggle.

To do this, go to the competition site on Kaggle at https://www.kaggle.com/t/185418aa7ed24db3b98ed851a4db2b41 and click on user then click on your profile as shown below. Click Account.

![alt text](https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/Capture-NLP.PNG)



### 2. Next, scroll down to the API access section and click on **Create New Token** to download an API key (kaggle.json).

![alt text](https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/Capture-NLP_1.PNG)

### 3. Upload your kaggle.json file using the following snippet in a code cell:



In [1]:
from google.colab import files
files.upload()

In [2]:
#If successfully uploaded in the above step, the 'ls' command here should display the kaggle.json file.
# %ls

### 4. Install the Kaggle API using the following command


In [3]:
!pip install -U -q kaggle==1.5.8

### 5. Move the kaggle.json file into ~/.kaggle, which is where the API client expects your token to be located:



In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
#Execute the following command to verify whether the kaggle.json is stored in the appropriate location: ~/.kaggle/kaggle.json
!ls ~/.kaggle

In [6]:
!chmod 600 /root/.kaggle/kaggle.json #run this command to ensure your Kaggle API token is secure on colab

### 6. Now download the Test Data from Kaggle

**NOTE: If you get a '403 - Not Found' error after running the cell below, it is most likely that the user (whose kaggle.json is uploaded above) has not 'accepted' the rules of the competition and therefore has 'not joined' the competition.**

If you encounter **401-unauthorised** download latest **kaggle.json** by repeating steps 1 & 2

In [7]:
#If you get a forbidden link, you have most likely not joined the competition.
!kaggle competitions download -c classification-of-caltech-256-images

In [8]:
!mkdir /content/Kaggle2Test
!unzip classification-of-caltech-256-images -d /content/Kaggle2Test/

In [9]:
!mkdir Kaggle2Test/test
!mv test/* Kaggle2Test/test/

### 7. Download the Train Data

In [10]:
%%capture
!wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/Caltech_256_Train.zip

!unzip "Caltech_256_Train.zip"

## Grading = 10 Marks

## YOUR CODING STARTS FROM HERE

### Import Required packages

In [1]:
# import the libraries
import numpy as np
import pandas as pd
import os,shutil,glob,PIL
import pathlib
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm
from collections import defaultdict
from collections import Counter

C:\Users\abhay\AppData\Roaming\Python\Python310\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


### **Stage 1:** Data Loading and preprocessing of Images (3 points)

#### Analyze the shape of images and distribution of classes

**The below two cells were run later (but shown here) after loading data with transform without normalization to get appropriate mean and standard deviation.**

In [12]:
# mean = 0.0
# for img, _ in train_data:
#     mean += img.mean([1,2])
# mean = mean/len(train_data)
# print(mean)

In [13]:
# sumel = 0.0
# countel = 0
# for img, _ in train_data:
#     img = (img - mean.unsqueeze(1).unsqueeze(1))**2
#     sumel += img.sum([1, 2])
#     countel += torch.numel(img[0])
# std = torch.sqrt(sumel/countel)
# print(std)

In [14]:
# Normalize with mean and std
# train_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize((0.4839, 0.4528, 0.3962), (0.2702, 0.2655, 0.2745))])
# train_transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
# transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor(), transforms.Normalize((0.5503, 0.5315, 0.5028), (0.3162, 0.3125, 0.3263))])

In [2]:
# Define train and test transforms for data preprocessing and image augmentation
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4839, 0.4528, 0.3962), (0.2702, 0.2655, 0.2745))])

test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4839, 0.4528, 0.3962), (0.2702, 0.2655, 0.2745))])

In [3]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


**We will be adjusting these values 224 224 after calculating mean and std dev. later on.**

In [4]:
# Loading the train set file
train_data_folder = "./classification-of-caltech-256-images/train" # Train directory for loading images
total_dataset = datasets.ImageFolder(root=train_data_folder, transform=train_transform)
# train_data = datasets.ImageFolder(root=train_data_folder)

In [5]:
# splitting between train and Validation set
train_size = int(0.9 * len(total_dataset))  # 90% for training
val_size = len(total_dataset) - train_size  # Remaining 10% for validation
train_data, val_data = torch.utils.data.random_split(total_dataset, [train_size, val_size])

In [6]:
#defiining train_batch_size
train_batch_size = 32

In [7]:
# Create data loaders for training and validation
train_loader = torch.utils.data.DataLoader(train_data, batch_size=train_batch_size,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=train_batch_size,shuffle=False)

In [8]:
# Loading the test set file
test_data_folder = "./classification-of-caltech-256-images/test_images" # Test directory for loading images
test_data = datasets.ImageFolder(root=test_data_folder, transform=test_transform)

In [9]:
test_data.classes
total_dataset.classes

['001.ak47',
 '002.american-flag',
 '003.backpack',
 '004.baseball-bat',
 '005.baseball-glove',
 '006.basketball-hoop',
 '007.bat',
 '008.bathtub',
 '009.bear',
 '010.beer-mug',
 '011.billiards',
 '012.binoculars',
 '013.birdbath',
 '014.blimp',
 '015.bonsai-101',
 '016.boom-box',
 '017.bowling-ball',
 '018.bowling-pin',
 '019.boxing-glove',
 '020.brain-101',
 '021.breadmaker',
 '022.buddha-101',
 '023.bulldozer',
 '024.butterfly',
 '025.cactus',
 '026.cake',
 '027.calculator',
 '028.camel',
 '029.cannon',
 '030.canoe',
 '031.car-tire',
 '032.cartman',
 '033.cd',
 '034.centipede',
 '035.cereal-box',
 '036.chandelier-101',
 '037.chess-board',
 '038.chimp',
 '039.chopsticks',
 '040.cockroach',
 '041.coffee-mug',
 '042.coffin',
 '043.coin',
 '044.comet',
 '045.computer-keyboard',
 '046.computer-monitor',
 '047.computer-mouse',
 '048.conch',
 '049.cormorant',
 '050.covered-wagon',
 '051.cowboy-hat',
 '052.crab-101',
 '053.desk-globe',
 '054.diamond-ring',
 '055.dice',
 '056.dog',
 '057.dol

In [10]:
# # Initializing batch size
# batch_size = 64

# # Loading the train dataset
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [10]:
# Generate a batches of images and labels
train_images, train_labels = next(iter(train_loader))
train_images.shape, train_labels.shape

(torch.Size([32, 3, 224, 224]), torch.Size([32]))

In [11]:
# labels Translator
label_names = {v: k for k, v in total_dataset.class_to_idx.items()}
label_names

In [26]:
# Create a grid of images along with their corresponding labels
L = 3
W = 3

fig, axes = plt.subplots(L, W, figsize = (10, 10))
axes = axes.reshape(-1)

for i in np.arange(0, L*W):
    axes[i].imshow(train_images[i].permute(1, 2, 0))
    axes[i].set_title(label_names[train_labels[i].item()])
    axes[i].axis('on')

plt.tight_layout()

In [27]:
mean = 0.0
for img, _ in train_data:
    mean += img.mean([1,2])
mean = mean/len(train_data)
print(mean)

In [28]:
sumel = 0.0
countel = 0
for img, _ in train_data:
    img = (img - mean.unsqueeze(1).unsqueeze(1))**2
    sumel += img.sum([1, 2])
    countel += torch.numel(img[0])
std = torch.sqrt(sumel/countel)
print(std)

In [12]:
num_classes = len(total_dataset.classes)

In [29]:
# sum = 0
# for label in train_data.classes:
#     num_img = len(train_data.targets[train_data.targets == train_data.class_to_idx[label]])
#     print (num_img)
# print (sum)

num_classes = len(total_dataset.classes)
dataset_size = len(total_dataset)
classes = total_dataset.classes
img_dict = {}
for i in range(num_classes):
    img_dict[classes[i]] = 0

for i in range(dataset_size):
    img, label = total_dataset[i]
    img_dict[classes[label]] += 1

# img_dict

**Our observation is all those minority classes we need to go for augmenting new samples.**

In [13]:
# No of Categories
print(len(total_dataset.classes))

257


In [31]:
# total_dataset.targets

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
# Calculate class weights
class_weights = torch.tensor([1.0] * num_classes)  # Initialize with equal weights

# Calculate class frequencies
class_counts = torch.bincount(torch.tensor(total_dataset.targets))
total_samples = len(total_dataset)
class_frequencies = class_counts.float() / total_samples

# Update class weights based on class frequencies
class_weights = 1.0 / class_frequencies

# Normalize class weights
class_weights /= torch.max(class_weights)
class_weights.to(device)

tensor([0.8163, 0.8247, 0.5298, 0.6299, 0.5405, 0.8889, 0.7547, 0.3448, 0.7843,
        0.8511, 0.2878, 0.3704, 0.8163, 0.9302, 0.6557, 0.8791, 0.7692, 0.7921,
        0.6452, 0.9639, 0.5634, 0.8247, 0.7273, 0.7143, 0.7018, 0.7547, 0.8000,
        0.7273, 0.7767, 0.7692, 0.8889, 0.7921, 0.7843, 0.8000, 0.9195, 0.7547,
        0.6667, 0.7273, 0.9412, 0.6452, 0.9195, 0.9195, 0.6452, 0.6612, 0.9412,
        0.6015, 0.8511, 0.7767, 0.7547, 0.8247, 0.7018, 0.9412, 0.9756, 0.6780,
        0.8163, 0.7843, 0.7547, 0.8602, 0.9639, 0.9195, 0.7843, 0.9639, 0.6557,
        0.6107, 0.7921, 0.9639, 0.9639, 0.7273, 0.8081, 0.9524, 0.8081, 0.6780,
        0.8000, 0.6957, 0.9639, 0.9524, 0.8696, 0.8889, 0.8081, 0.6897, 0.8421,
        0.9877, 0.8421, 0.9524, 0.7143, 1.0000, 0.8602, 0.8163, 0.7273, 0.3774,
        0.8421, 0.3980, 0.7143, 0.7692, 0.9302, 0.2807, 0.8989, 0.8000, 1.0000,
        0.8602, 0.5797, 0.9091, 0.7207, 0.8247, 0.2963, 0.9195, 0.8989, 0.9412,
        0.5128, 0.9412, 0.9524, 0.9524, 

In [16]:
print(device)

cuda


In [17]:
# Number of training samples
print(len(total_dataset))

30607


In [18]:
# Size of one training image
print(train_data[0][0].size(), val_data[0][0].size())

torch.Size([3, 224, 224]) torch.Size([3, 224, 224])


In [21]:
# max(img_dict,key=img_dict.get)

In [38]:
# print(img_dict.values())
# total = sum(img_dict.values())

In [19]:
# To test whether GPU instance is present in the system of not.
use_cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, 'CUDA:', use_cuda)

Using PyTorch version: 1.12.1+cu116 CUDA: True


### **Stage 2:** Build and train the CNN model using Keras/Pytorch (5 points)

You can train the CNN model and Pre-trained model and then compare the model performance on the kaggle testset


### Transfer learning

Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.

A pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classification task.

The intuition behind transfer learning for image classification is that if a model is trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world. You can then take advantage of these learned feature maps without having to start from scratch by training a large model on a large dataset.



In [47]:
torch.cuda.empty_cache()

<b> Creating and using the Native CNN model using BatchNorm, Pool and skip connections <\b>

In [ ]:
class CaltechCNN(nn.Module):
    def __init__(self, num_classes):
      super(CaltechCNN, self).__init__()
      self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
      self.bn1 = nn.BatchNorm2d(64)
      self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
      self.bn2 = nn.BatchNorm2d(128)
      self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
      self.bn3 = nn.BatchNorm2d(256)
      self.pool = nn.MaxPool2d(2, 2)
      self.skip_conv1 = nn.Conv2d(64, 128, kernel_size=1, stride=1)
      self.skip_conv2 = nn.Conv2d(128, 256, kernel_size=1, stride=1)
      self.fc1 = nn.Linear(256 * 8 * 8, 512)
      self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
      x = self.conv1(x)
      x = self.bn1(x)
      x = F.relu(x)
      x = self.pool(x)
      skip = self.skip_conv1(x)
      x = self.conv2(x)
      x = self.bn2(x)
      x = F.relu(x)
      x = self.pool(x)
      skip = self.skip_conv2(skip)
      x += F.interpolate(skip, size=x.size()[2:], mode='bilinear', align_corners=False)
      x = F.relu(x)
      x = self.conv3(x)
      x = self.bn3(x)
      x = F.relu(x)
      x = self.pool(x)
      x = x.view(-1, 256 * 8 * 8)
      x = F.relu(self.fc1(x))
      x = self.fc2(x)
      return x

In [ ]:
model_ft = CaltechCNN(num_classes=num_classes)
model_ft = model_ft.to(device)
print(model_ft)

#### Use the pre-trained models

* Load the pre-trained model
* Train and evaluate the images

<b> Now using Pretrained GoogleNet model <\b>

In [20]:
model_ft = models.googlenet(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes)
#model.softmax = nn.softmax()
model_ft = model_ft.to(device)
print(model_ft)

C:\Users\abhay\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\abhay\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [21]:
## use this cell to perform incremental training of the saved models:

# PATH='CalTech-256-GoogleNet_v307_2307.pth'
# model_ft.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [22]:
criterion = nn.CrossEntropyLoss(weight=class_weights).to(device)
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)

In [23]:
epsilon = 0.005  # Perturbation strength for FGSM

In [24]:
train_accu = []     # Empty list for saving train accuracy
train_losses = []   # Empty list for saving train losses
val_accu = []

In [ ]:
# Tried making the below training an Adversarial Robustness with FGSM

In [31]:
def train(epoch):
  print('\nEpoch : %d'%epoch)

  model_ft.train()    # Initiate the model in training mode

  running_loss = 0
  correct = 0
  total = 0

  for data in tqdm(train_loader):

    inputs,labels=data[0].to(device),data[1].to(device)   # Loading the input tensors into CUDA GPU
    # print(inputs,labels)
    # Generate adversarial examples using FGSM
    # inputs.requires_grad = True

    optimizer.zero_grad()
    outputs = model_ft(inputs)
    loss = criterion(outputs,labels)  # Calculating the loss
    loss.backward()                   # Back Propagation for calculaing gradients and adjusting weights

    # Create perturbations based on the sign of gradients
    # inputs_grad = inputs.grad.data
    # perturbed_inputs = torch.clamp(inputs + epsilon * torch.sign(inputs_grad), 0, 1)
    # perturbed_inputs = perturbed_inputs.detach()

    # # Perform forward pass with perturbed inputs
    # outputs = model_ft(perturbed_inputs)
    # loss = criterion(outputs, labels)
    # loss.backward()

    optimizer.step()

    running_loss += loss.item()

    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()




  train_loss = running_loss/len(train_loader)     # Calculating the mean of training loss
  accu = 100.*correct/total                       # Calculating the accuracy

  train_accu.append(accu)
  train_losses.append(train_loss)
  print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))

<b>The below Eval_Val function also outputs Top N wrong predictions against their Ground Truth Label</b>

In [32]:
  # Validation
def eval_val(epoch):
    model_ft.eval()
    wrong_predictions = []
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
      for i, data in enumerate(val_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)

        # Find wrong predictions
        for j in range(len(predicted)):
          if predicted[j] != labels[j]:
              wrong_predictions.append({
                  'input': inputs[j],
                  'predicted_label': predicted[j].item(),
                  'true_label': labels[j].item()
              })
        # print(val_loss)
        val_loss += loss.item()
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()


    val_batch_loss = val_loss/len(val_loader)
    val_batch_accu = 100.*correct/total                       # Calculating the accuracy
    val_accu.append(val_batch_accu)
    print('Val Loss: %.3f| Accuracy: %.3f' %(val_batch_loss,val_batch_accu))

    print('Top 5 Wrong Predictions in Validation Set:')
    counter = Counter([prediction['true_label'] for prediction in wrong_predictions])
    top_wrong_predictions = counter.most_common(5)

    for true_label, count in top_wrong_predictions:
        print(f'True Label: {true_label}, Count: {count}')
        filtered_predictions = [prediction for prediction in wrong_predictions
                                if prediction['true_label'] == true_label][:5]
        for prediction in filtered_predictions:
            input_image = prediction['input']
            predicted_label = prediction['predicted_label']
            true_label = prediction['true_label']
            print(f'Predicted Label: {predicted_label}, True Label: {true_label}')


    # # Print details of wrong predictions
    # print('Wrong Predictions in Validation Set:')
    # for prediction in wrong_predictions:
    #     input_image = prediction['input']
    #     predicted_label = prediction['predicted_label']
    #     true_label = prediction['true_label']
    #     print(f'Predicted Label: {predicted_label}, True Label: {true_label}')
    #     # Additional code to visualize or process the input image if needed


In [42]:
epochs = 12 # 20 run modify it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)


Epoch : 1


100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:49<00:00,  2.10it/s]


Train Loss: 0.508 | Accuracy: 86.390
Val Loss: 0.613| Accuracy: 84.156
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 34
Predicted Label: 25, True Label: 256
Predicted Label: 232, True Label: 256
Predicted Label: 202, True Label: 256
Predicted Label: 89, True Label: 256
Predicted Label: 91, True Label: 256
True Label: 125, Count: 14
Predicted Label: 134, True Label: 125
Predicted Label: 228, True Label: 125
Predicted Label: 149, True Label: 125
Predicted Label: 209, True Label: 125
Predicted Label: 256, True Label: 125
True Label: 231, Count: 10
Predicted Label: 234, True Label: 231
Predicted Label: 116, True Label: 231
Predicted Label: 158, True Label: 231
Predicted Label: 222, True Label: 231
Predicted Label: 217, True Label: 231
True Label: 39, Count: 9
Predicted Label: 197, True Label: 39
Predicted Label: 98, True Label: 39
Predicted Label: 242, True Label: 39
Predicted Label: 176, True Label: 39
Predicted Label: 4, True Label: 39
True Label: 146, Count: 9
Predi

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [13:08<00:00,  1.09it/s]


Train Loss: 0.514 | Accuracy: 86.172
Val Loss: 0.672| Accuracy: 82.881
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 30
Predicted Label: 103, True Label: 256
Predicted Label: 232, True Label: 256
Predicted Label: 79, True Label: 256
Predicted Label: 67, True Label: 256
Predicted Label: 53, True Label: 256
True Label: 125, Count: 13
Predicted Label: 228, True Label: 125
Predicted Label: 21, True Label: 125
Predicted Label: 246, True Label: 125
Predicted Label: 84, True Label: 125
Predicted Label: 158, True Label: 125
True Label: 104, Count: 12
Predicted Label: 118, True Label: 104
Predicted Label: 28, True Label: 104
Predicted Label: 57, True Label: 104
Predicted Label: 63, True Label: 104
Predicted Label: 118, True Label: 104
True Label: 146, Count: 11
Predicted Label: 118, True Label: 146
Predicted Label: 18, True Label: 146
Predicted Label: 212, True Label: 146
Predicted Label: 84, True Label: 146
Predicted Label: 40, True Label: 146
True Label: 231, Count: 10
Pr

...

Train Loss: 0.481 | Accuracy: 87.029
Val Loss: 0.623| Accuracy: 83.763
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 40
Predicted Label: 25, True Label: 256
Predicted Label: 159, True Label: 256
Predicted Label: 160, True Label: 256
Predicted Label: 89, True Label: 256
Predicted Label: 8, True Label: 256
True Label: 250, Count: 14
Predicted Label: 74, True Label: 250
Predicted Label: 68, True Label: 250
Predicted Label: 68, True Label: 250
Predicted Label: 68, True Label: 250
Predicted Label: 76, True Label: 250
True Label: 125, Count: 12
Predicted Label: 250, True Label: 125
Predicted Label: 172, True Label: 125
Predicted Label: 186, True Label: 125
Predicted Label: 246, True Label: 125
Predicted Label: 175, True Label: 125
True Label: 231, Count: 10
Predicted Label: 11, True Label: 231
Predicted Label: 100, True Label: 231
Predicted Label: 158, True Label: 231
Predicted Label: 149, True Label: 231
Predicted Label: 204, True Label: 231
True Label: 95, Count: 10
Pr

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:43<00:00,  2.13it/s]


Train Loss: 0.500 | Accuracy: 86.572
Val Loss: 0.674| Accuracy: 82.849
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 31
Predicted Label: 160, True Label: 256
Predicted Label: 233, True Label: 256
Predicted Label: 6, True Label: 256
Predicted Label: 91, True Label: 256
Predicted Label: 67, True Label: 256
True Label: 125, Count: 15
Predicted Label: 228, True Label: 125
Predicted Label: 172, True Label: 125
Predicted Label: 156, True Label: 125
Predicted Label: 6, True Label: 125
Predicted Label: 230, True Label: 125
True Label: 146, Count: 11
Predicted Label: 12, True Label: 146
Predicted Label: 158, True Label: 146
Predicted Label: 220, True Label: 146
Predicted Label: 193, True Label: 146
Predicted Label: 188, True Label: 146
True Label: 250, Count: 11
Predicted Label: 172, True Label: 250
Predicted Label: 132, True Label: 250
Predicted Label: 68, True Label: 250
Predicted Label: 68, True Label: 250
Predicted Label: 99, True Label: 250
True Label: 231, Count: 10
P

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:47<00:00,  2.11it/s]


Train Loss: 0.488 | Accuracy: 86.880
Val Loss: 0.674| Accuracy: 83.045
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 35
Predicted Label: 25, True Label: 256
Predicted Label: 142, True Label: 256
Predicted Label: 236, True Label: 256
Predicted Label: 12, True Label: 256
Predicted Label: 220, True Label: 256
True Label: 125, Count: 12
Predicted Label: 158, True Label: 125
Predicted Label: 7, True Label: 125
Predicted Label: 147, True Label: 125
Predicted Label: 230, True Label: 125
Predicted Label: 228, True Label: 125
True Label: 104, Count: 12
Predicted Label: 115, True Label: 104
Predicted Label: 85, True Label: 104
Predicted Label: 133, True Label: 104
Predicted Label: 194, True Label: 104
Predicted Label: 143, True Label: 104
True Label: 146, Count: 11
Predicted Label: 200, True Label: 146
Predicted Label: 158, True Label: 146
Predicted Label: 220, True Label: 146
Predicted Label: 205, True Label: 146
Predicted Label: 193, True Label: 146
True Label: 250, Count:

In [33]:
epochs = 12 # 20 run modify it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)


Epoch : 1


100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [10:03<00:00,  1.43it/s]


Train Loss: 0.594 | Accuracy: 84.070
Val Loss: 0.432| Accuracy: 88.697
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 35
Predicted Label: 190, True Label: 256
Predicted Label: 160, True Label: 256
Predicted Label: 173, True Label: 256
Predicted Label: 6, True Label: 256
Predicted Label: 91, True Label: 256
True Label: 125, Count: 10
Predicted Label: 58, True Label: 125
Predicted Label: 7, True Label: 125
Predicted Label: 230, True Label: 125
Predicted Label: 230, True Label: 125
Predicted Label: 97, True Label: 125
True Label: 95, Count: 9
Predicted Label: 206, True Label: 95
Predicted Label: 180, True Label: 95
Predicted Label: 148, True Label: 95
Predicted Label: 228, True Label: 95
Predicted Label: 29, True Label: 95
True Label: 7, Count: 7
Predicted Label: 130, True Label: 7
Predicted Label: 158, True Label: 7
Predicted Label: 20, True Label: 7
Predicted Label: 101, True Label: 7
Predicted Label: 190, True Label: 7
True Label: 104, Count: 7
Predicted Label: 49, 

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [18:09<00:00,  1.27s/it]


Train Loss: 0.573 | Accuracy: 84.568
Val Loss: 0.488| Accuracy: 87.226
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 23
Predicted Label: 25, True Label: 256
Predicted Label: 141, True Label: 256
Predicted Label: 91, True Label: 256
Predicted Label: 149, True Label: 256
Predicted Label: 197, True Label: 256
True Label: 104, Count: 9
Predicted Label: 49, True Label: 104
Predicted Label: 49, True Label: 104
Predicted Label: 164, True Label: 104
Predicted Label: 49, True Label: 104
Predicted Label: 6, True Label: 104
True Label: 250, Count: 7
Predicted Label: 95, True Label: 250
Predicted Label: 101, True Label: 250
Predicted Label: 68, True Label: 250
Predicted Label: 99, True Label: 250
Predicted Label: 196, True Label: 250
True Label: 125, Count: 6
Predicted Label: 70, True Label: 125
Predicted Label: 84, True Label: 125
Predicted Label: 221, True Label: 125
Predicted Label: 207, True Label: 125
Predicted Label: 164, True Label: 125
True Label: 23, Count: 5
Predicte

...

Train Loss: 0.526 | Accuracy: 85.726
Val Loss: 0.615| Accuracy: 83.698
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 37
Predicted Label: 25, True Label: 256
Predicted Label: 132, True Label: 256
Predicted Label: 137, True Label: 256
Predicted Label: 137, True Label: 256
Predicted Label: 79, True Label: 256
True Label: 125, Count: 12
Predicted Label: 3, True Label: 125
Predicted Label: 96, True Label: 125
Predicted Label: 147, True Label: 125
Predicted Label: 244, True Label: 125
Predicted Label: 158, True Label: 125
True Label: 231, Count: 11
Predicted Label: 9, True Label: 231
Predicted Label: 100, True Label: 231
Predicted Label: 158, True Label: 231
Predicted Label: 18, True Label: 231
Predicted Label: 166, True Label: 231
True Label: 104, Count: 10
Predicted Label: 100, True Label: 104
Predicted Label: 28, True Label: 104
Predicted Label: 144, True Label: 104
Predicted Label: 143, True Label: 104
Predicted Label: 27, True Label: 104
True Label: 146, Count: 8
Pr

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:40<00:00,  2.15it/s]


Train Loss: 0.515 | Accuracy: 85.991
Val Loss: 0.577| Accuracy: 84.711
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 32
Predicted Label: 160, True Label: 256
Predicted Label: 147, True Label: 256
Predicted Label: 6, True Label: 256
Predicted Label: 91, True Label: 256
Predicted Label: 67, True Label: 256
True Label: 231, Count: 11
Predicted Label: 21, True Label: 231
Predicted Label: 118, True Label: 231
Predicted Label: 74, True Label: 231
Predicted Label: 216, True Label: 231
Predicted Label: 34, True Label: 231
True Label: 146, Count: 9
Predicted Label: 158, True Label: 146
Predicted Label: 51, True Label: 146
Predicted Label: 86, True Label: 146
Predicted Label: 147, True Label: 146
Predicted Label: 63, True Label: 146
True Label: 104, Count: 8
Predicted Label: 244, True Label: 104
Predicted Label: 88, True Label: 104
Predicted Label: 63, True Label: 104
Predicted Label: 39, True Label: 104
Predicted Label: 125, True Label: 104
True Label: 250, Count: 7
Predict

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:42<00:00,  2.14it/s]


Train Loss: 0.507 | Accuracy: 86.052
Val Loss: 0.595| Accuracy: 84.842
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 25
Predicted Label: 25, True Label: 256
Predicted Label: 91, True Label: 256
Predicted Label: 214, True Label: 256
Predicted Label: 149, True Label: 256
Predicted Label: 198, True Label: 256
True Label: 125, Count: 14
Predicted Label: 206, True Label: 125
Predicted Label: 158, True Label: 125
Predicted Label: 255, True Label: 125
Predicted Label: 192, True Label: 125
Predicted Label: 158, True Label: 125
True Label: 146, Count: 9
Predicted Label: 240, True Label: 146
Predicted Label: 158, True Label: 146
Predicted Label: 149, True Label: 146
Predicted Label: 179, True Label: 146
Predicted Label: 188, True Label: 146
True Label: 10, Count: 8
Predicted Label: 226, True Label: 10
Predicted Label: 3, True Label: 10
Predicted Label: 125, True Label: 10
Predicted Label: 158, True Label: 10
Predicted Label: 158, True Label: 10
True Label: 7, Count: 8
Predic

In [30]:
epochs = 10 # 20 run modify it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)


Epoch : 1


100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [11:15<00:00,  1.27it/s]


Train Loss: 0.635 | Accuracy: 83.293
Val Loss: 0.478| Accuracy: 86.704
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 39
Predicted Label: 221, True Label: 256
Predicted Label: 116, True Label: 256
Predicted Label: 161, True Label: 256
Predicted Label: 38, True Label: 256
Predicted Label: 167, True Label: 256
True Label: 95, Count: 10
Predicted Label: 17, True Label: 95
Predicted Label: 41, True Label: 95
Predicted Label: 24, True Label: 95
Predicted Label: 143, True Label: 95
Predicted Label: 196, True Label: 95
True Label: 125, Count: 9
Predicted Label: 0, True Label: 125
Predicted Label: 49, True Label: 125
Predicted Label: 67, True Label: 125
Predicted Label: 244, True Label: 125
Predicted Label: 84, True Label: 125
True Label: 250, Count: 9
Predicted Label: 56, True Label: 250
Predicted Label: 62, True Label: 250
Predicted Label: 177, True Label: 250
Predicted Label: 68, True Label: 250
Predicted Label: 177, True Label: 250
True Label: 104, Count: 9
Predicted La

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [23:01<00:00,  1.60s/it]


Train Loss: 0.624 | Accuracy: 83.362
Val Loss: 0.561| Accuracy: 85.430
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 30
Predicted Label: 161, True Label: 256
Predicted Label: 41, True Label: 256
Predicted Label: 6, True Label: 256
Predicted Label: 205, True Label: 256
Predicted Label: 187, True Label: 256
True Label: 125, Count: 13
Predicted Label: 29, True Label: 125
Predicted Label: 243, True Label: 125
Predicted Label: 149, True Label: 125
Predicted Label: 204, True Label: 125
Predicted Label: 62, True Label: 125
True Label: 158, Count: 11
Predicted Label: 231, True Label: 158
Predicted Label: 127, True Label: 158
Predicted Label: 34, True Label: 158
Predicted Label: 21, True Label: 158
Predicted Label: 253, True Label: 158
True Label: 95, Count: 9
Predicted Label: 107, True Label: 95
Predicted Label: 212, True Label: 95
Predicted Label: 194, True Label: 95
Predicted Label: 118, True Label: 95
Predicted Label: 108, True Label: 95
True Label: 7, Count: 8
Predicte

...

Train Loss: 0.574 | Accuracy: 84.597
Val Loss: 0.651| Accuracy: 82.849
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 33
Predicted Label: 238, True Label: 256
Predicted Label: 6, True Label: 256
Predicted Label: 71, True Label: 256
Predicted Label: 125, True Label: 256
Predicted Label: 24, True Label: 256
True Label: 125, Count: 14
Predicted Label: 244, True Label: 125
Predicted Label: 177, True Label: 125
Predicted Label: 25, True Label: 125
Predicted Label: 61, True Label: 125
Predicted Label: 24, True Label: 125
True Label: 104, Count: 13
Predicted Label: 169, True Label: 104
Predicted Label: 166, True Label: 104
Predicted Label: 235, True Label: 104
Predicted Label: 56, True Label: 104
Predicted Label: 166, True Label: 104
True Label: 95, Count: 12
Predicted Label: 229, True Label: 95
Predicted Label: 230, True Label: 95
Predicted Label: 34, True Label: 95
Predicted Label: 153, True Label: 95
Predicted Label: 164, True Label: 95
True Label: 190, Count: 10
Predic

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [05:33<00:00,  2.58it/s]


Train Loss: 0.570 | Accuracy: 84.829
Val Loss: 0.650| Accuracy: 83.012
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 43
Predicted Label: 28, True Label: 256
Predicted Label: 225, True Label: 256
Predicted Label: 22, True Label: 256
Predicted Label: 167, True Label: 256
Predicted Label: 114, True Label: 256
True Label: 95, Count: 20
Predicted Label: 148, True Label: 95
Predicted Label: 41, True Label: 95
Predicted Label: 127, True Label: 95
Predicted Label: 213, True Label: 95
Predicted Label: 196, True Label: 95
True Label: 158, Count: 11
Predicted Label: 104, True Label: 158
Predicted Label: 127, True Label: 158
Predicted Label: 252, True Label: 158
Predicted Label: 171, True Label: 158
Predicted Label: 159, True Label: 158
True Label: 125, Count: 11
Predicted Label: 82, True Label: 125
Predicted Label: 2, True Label: 125
Predicted Label: 71, True Label: 125
Predicted Label: 156, True Label: 125
Predicted Label: 198, True Label: 125
True Label: 25, Count: 8
Predic

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [05:13<00:00,  2.74it/s]


Train Loss: 0.562 | Accuracy: 84.883
Val Loss: 0.645| Accuracy: 82.816
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 43
Predicted Label: 72, True Label: 256
Predicted Label: 91, True Label: 256
Predicted Label: 185, True Label: 256
Predicted Label: 132, True Label: 256
Predicted Label: 127, True Label: 256
True Label: 95, Count: 13
Predicted Label: 41, True Label: 95
Predicted Label: 158, True Label: 95
Predicted Label: 153, True Label: 95
Predicted Label: 158, True Label: 95
Predicted Label: 161, True Label: 95
True Label: 104, Count: 9
Predicted Label: 161, True Label: 104
Predicted Label: 29, True Label: 104
Predicted Label: 213, True Label: 104
Predicted Label: 63, True Label: 104
Predicted Label: 175, True Label: 104
True Label: 228, Count: 9
Predicted Label: 131, True Label: 228
Predicted Label: 129, True Label: 228
Predicted Label: 70, True Label: 228
Predicted Label: 141, True Label: 228
Predicted Label: 145, True Label: 228
True Label: 157, Count: 8
Predic

In [37]:
epochs = 10 # 20 run modify it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)


Epoch : 1


100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:39<00:00,  2.16it/s]


Train Loss: 0.745 | Accuracy: 80.237
Val Loss: 0.623| Accuracy: 83.698
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 37
Predicted Label: 240, True Label: 256
Predicted Label: 193, True Label: 256
Predicted Label: 233, True Label: 256
Predicted Label: 8, True Label: 256
Predicted Label: 233, True Label: 256
True Label: 125, Count: 16
Predicted Label: 127, True Label: 125
Predicted Label: 146, True Label: 125
Predicted Label: 191, True Label: 125
Predicted Label: 180, True Label: 125
Predicted Label: 244, True Label: 125
True Label: 158, Count: 12
Predicted Label: 114, True Label: 158
Predicted Label: 104, True Label: 158
Predicted Label: 118, True Label: 158
Predicted Label: 252, True Label: 158
Predicted Label: 37, True Label: 158
True Label: 104, Count: 9
Predicted Label: 95, True Label: 104
Predicted Label: 49, True Label: 104
Predicted Label: 29, True Label: 104
Predicted Label: 60, True Label: 104
Predicted Label: 115, True Label: 104
True Label: 95, Count: 8
P

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [10:17<00:00,  1.39it/s]


Train Loss: 0.718 | Accuracy: 80.926
Val Loss: 0.673| Accuracy: 83.567
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 31
Predicted Label: 216, True Label: 256
Predicted Label: 242, True Label: 256
Predicted Label: 88, True Label: 256
Predicted Label: 161, True Label: 256
Predicted Label: 97, True Label: 256
True Label: 125, Count: 17
Predicted Label: 170, True Label: 125
Predicted Label: 255, True Label: 125
Predicted Label: 121, True Label: 125
Predicted Label: 191, True Label: 125
Predicted Label: 180, True Label: 125
True Label: 104, Count: 13
Predicted Label: 121, True Label: 104
Predicted Label: 49, True Label: 104
Predicted Label: 95, True Label: 104
Predicted Label: 69, True Label: 104
Predicted Label: 147, True Label: 104
True Label: 158, Count: 12
Predicted Label: 204, True Label: 158
Predicted Label: 134, True Label: 158
Predicted Label: 70, True Label: 158
Predicted Label: 157, True Label: 158
Predicted Label: 100, True Label: 158
True Label: 140, Count: 

...

Train Loss: 0.674 | Accuracy: 81.968
Val Loss: 0.726| Accuracy: 81.575
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 36
Predicted Label: 193, True Label: 256
Predicted Label: 74, True Label: 256
Predicted Label: 200, True Label: 256
Predicted Label: 216, True Label: 256
Predicted Label: 97, True Label: 256
True Label: 189, Count: 11
Predicted Label: 197, True Label: 189
Predicted Label: 165, True Label: 189
Predicted Label: 178, True Label: 189
Predicted Label: 165, True Label: 189
Predicted Label: 212, True Label: 189
True Label: 158, Count: 11
Predicted Label: 109, True Label: 158
Predicted Label: 192, True Label: 158
Predicted Label: 206, True Label: 158
Predicted Label: 58, True Label: 158
Predicted Label: 95, True Label: 158
True Label: 231, Count: 10
Predicted Label: 20, True Label: 231
Predicted Label: 143, True Label: 231
Predicted Label: 143, True Label: 231
Predicted Label: 96, True Label: 231
Predicted Label: 19, True Label: 231
True Label: 95, Count: 10

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:43<00:00,  2.13it/s]


Train Loss: 0.661 | Accuracy: 82.237
Val Loss: 0.876| Accuracy: 78.406
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 37
Predicted Label: 67, True Label: 256
Predicted Label: 123, True Label: 256
Predicted Label: 63, True Label: 256
Predicted Label: 73, True Label: 256
Predicted Label: 205, True Label: 256
True Label: 95, Count: 15
Predicted Label: 161, True Label: 95
Predicted Label: 172, True Label: 95
Predicted Label: 168, True Label: 95
Predicted Label: 216, True Label: 95
Predicted Label: 191, True Label: 95
True Label: 125, Count: 15
Predicted Label: 170, True Label: 125
Predicted Label: 243, True Label: 125
Predicted Label: 28, True Label: 125
Predicted Label: 180, True Label: 125
Predicted Label: 58, True Label: 125
True Label: 158, Count: 12
Predicted Label: 204, True Label: 158
Predicted Label: 243, True Label: 158
Predicted Label: 37, True Label: 158
Predicted Label: 222, True Label: 158
Predicted Label: 170, True Label: 158
True Label: 104, Count: 11
Pre

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [06:40<00:00,  2.15it/s]


Train Loss: 0.653 | Accuracy: 82.720
Val Loss: 0.785| Accuracy: 79.288
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 33
Predicted Label: 240, True Label: 256
Predicted Label: 205, True Label: 256
Predicted Label: 82, True Label: 256
Predicted Label: 24, True Label: 256
Predicted Label: 133, True Label: 256
True Label: 104, Count: 19
Predicted Label: 63, True Label: 104
Predicted Label: 60, True Label: 104
Predicted Label: 231, True Label: 104
Predicted Label: 49, True Label: 104
Predicted Label: 161, True Label: 104
True Label: 125, Count: 14
Predicted Label: 149, True Label: 125
Predicted Label: 168, True Label: 125
Predicted Label: 180, True Label: 125
Predicted Label: 102, True Label: 125
Predicted Label: 213, True Label: 125
True Label: 231, Count: 12
Predicted Label: 1, True Label: 231
Predicted Label: 193, True Label: 231
Predicted Label: 147, True Label: 231
Predicted Label: 209, True Label: 231
Predicted Label: 93, True Label: 231
True Label: 158, Count: 11

In [54]:
epochs = 10 # 20 run modify it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)


Epoch : 1


100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [11:25<00:00,  1.26it/s]


Train Loss: 0.666 | Accuracy: 82.458
Val Loss: 0.812| Accuracy: 79.320
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 36
Predicted Label: 79, True Label: 256
Predicted Label: 123, True Label: 256
Predicted Label: 250, True Label: 256
Predicted Label: 93, True Label: 256
Predicted Label: 187, True Label: 256
True Label: 158, Count: 13
Predicted Label: 134, True Label: 158
Predicted Label: 25, True Label: 158
Predicted Label: 161, True Label: 158
Predicted Label: 125, True Label: 158
Predicted Label: 219, True Label: 158
True Label: 125, Count: 13
Predicted Label: 24, True Label: 125
Predicted Label: 104, True Label: 125
Predicted Label: 164, True Label: 125
Predicted Label: 232, True Label: 125
Predicted Label: 182, True Label: 125
True Label: 231, Count: 11
Predicted Label: 143, True Label: 231
Predicted Label: 74, True Label: 231
Predicted Label: 162, True Label: 231
Predicted Label: 137, True Label: 231
Predicted Label: 34, True Label: 231
True Label: 104, Count: 

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [10:15<00:00,  1.40it/s]


Train Loss: 0.644 | Accuracy: 82.640
Val Loss: 0.815| Accuracy: 79.451
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 35
Predicted Label: 161, True Label: 256
Predicted Label: 123, True Label: 256
Predicted Label: 70, True Label: 256
Predicted Label: 97, True Label: 256
Predicted Label: 141, True Label: 256
True Label: 104, Count: 15
Predicted Label: 253, True Label: 104
Predicted Label: 55, True Label: 104
Predicted Label: 158, True Label: 104
Predicted Label: 158, True Label: 104
Predicted Label: 118, True Label: 104
True Label: 125, Count: 13
Predicted Label: 161, True Label: 125
Predicted Label: 146, True Label: 125
Predicted Label: 180, True Label: 125
Predicted Label: 206, True Label: 125
Predicted Label: 7, True Label: 125
True Label: 158, Count: 12
Predicted Label: 204, True Label: 158
Predicted Label: 253, True Label: 158
Predicted Label: 143, True Label: 158
Predicted Label: 63, True Label: 158
Predicted Label: 143, True Label: 158
True Label: 95, Count: 1

...

Train Loss: 0.614 | Accuracy: 83.602
Val Loss: 0.906| Accuracy: 78.177
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 38
Predicted Label: 240, True Label: 256
Predicted Label: 123, True Label: 256
Predicted Label: 242, True Label: 256
Predicted Label: 213, True Label: 256
Predicted Label: 3, True Label: 256
True Label: 158, Count: 17
Predicted Label: 104, True Label: 158
Predicted Label: 162, True Label: 158
Predicted Label: 143, True Label: 158
Predicted Label: 116, True Label: 158
Predicted Label: 162, True Label: 158
True Label: 104, Count: 12
Predicted Label: 158, True Label: 104
Predicted Label: 55, True Label: 104
Predicted Label: 253, True Label: 104
Predicted Label: 64, True Label: 104
Predicted Label: 164, True Label: 104
True Label: 125, Count: 12
Predicted Label: 27, True Label: 125
Predicted Label: 104, True Label: 125
Predicted Label: 180, True Label: 125
Predicted Label: 116, True Label: 125
Predicted Label: 61, True Label: 125
True Label: 189, Count: 

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [22:14<00:00,  1.55s/it]


Train Loss: 0.612 | Accuracy: 83.994
Val Loss: 0.879| Accuracy: 78.928
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 35
Predicted Label: 216, True Label: 256
Predicted Label: 112, True Label: 256
Predicted Label: 123, True Label: 256
Predicted Label: 153, True Label: 256
Predicted Label: 208, True Label: 256
True Label: 158, Count: 15
Predicted Label: 172, True Label: 158
Predicted Label: 162, True Label: 158
Predicted Label: 228, True Label: 158
Predicted Label: 164, True Label: 158
Predicted Label: 222, True Label: 158
True Label: 104, Count: 14
Predicted Label: 146, True Label: 104
Predicted Label: 49, True Label: 104
Predicted Label: 121, True Label: 104
Predicted Label: 180, True Label: 104
Predicted Label: 172, True Label: 104
True Label: 84, Count: 12
Predicted Label: 133, True Label: 84
Predicted Label: 253, True Label: 84
Predicted Label: 206, True Label: 84
Predicted Label: 104, True Label: 84
Predicted Label: 3, True Label: 84
True Label: 95, Count: 12
P

100%|████████████████████████████████████████████████████████████████████████████████| 861/861 [05:33<00:00,  2.58it/s]


Train Loss: 0.611 | Accuracy: 83.660
Val Loss: 0.861| Accuracy: 77.883
Top 5 Wrong Predictions in Validation Set:
True Label: 256, Count: 34
Predicted Label: 206, True Label: 256
Predicted Label: 196, True Label: 256
Predicted Label: 213, True Label: 256
Predicted Label: 73, True Label: 256
Predicted Label: 205, True Label: 256
True Label: 158, Count: 17
Predicted Label: 114, True Label: 158
Predicted Label: 104, True Label: 158
Predicted Label: 192, True Label: 158
Predicted Label: 162, True Label: 158
Predicted Label: 228, True Label: 158
True Label: 231, Count: 13
Predicted Label: 20, True Label: 231
Predicted Label: 110, True Label: 231
Predicted Label: 2, True Label: 231
Predicted Label: 166, True Label: 231
Predicted Label: 164, True Label: 231
True Label: 104, Count: 13
Predicted Label: 158, True Label: 104
Predicted Label: 18, True Label: 104
Predicted Label: 253, True Label: 104
Predicted Label: 14, True Label: 104
Predicted Label: 175, True Label: 104
True Label: 95, Count: 1

In [49]:
# epochs = 30 # 20 run modify it later on
# for epoch in range(1, epochs+1):
#   train(epoch)
#   eval_val(epoch)

In [50]:
# epochs = 5 # 20 run moidfy it later on
# for epoch in range(1, epochs+1):
#   train(epoch)
#   eval_val(epoch)

In [51]:
# epochs = 3 # 20 run modify it later on
# for epoch in range(1, epochs+1):
#   train(epoch)
#   eval_val(epoch)

In [52]:
# epochs = 28 # 20 run modify it later on
# for epoch in range(1, epochs+1):
#   train(epoch)
#   eval_val(epoch)

Validatation data sets as well - todo

Hyper Parameters = epcoh, learning rate,

In [43]:
PATH = 'CalTech-256-GoogleNet_v307_2307.pth'
torch.save(model_ft.state_dict(), PATH)

In [ ]:
plt.plot(train_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train'])
plt.show()

In [ ]:
plt.plot(train_losses,'-o')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.legend(['Train'])
plt.title('Train Loss')
plt.show()

###   **Stage 3**: Evaluate the Model and get model predictions on the Kaggle testset (2 Points)









In [ ]:
PATH='CalTech-256-GoogleNet_v307_2307.pth'
model_ft.load_state_dict(torch.load(PATH))

In [44]:
model_ft.eval()

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [45]:
# Loading the train dataset
test_batch_size = 1
test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_batch_size,shuffle=False)

In [46]:
predictions = []
for i,data in enumerate(test_loader):
  input,_ = data
  output = model_ft(input.to(device))
  # print(output)
  pred = torch.max(output,dim=1)
  # print(pred)
  predictions.append(total_dataset.classes[pred.indices])

In [ ]:
# print([(x[0].split("/",3)[2]).split('\\',1)[1].replace('\\','/') for x in list(test_loader.dataset.imgs)])
# test_loader.dataset.imgs

In [47]:
results = pd.DataFrame()
results['img_path'] = pd.Series([(x[0].split("/",3)[2]).split('\\',1)[1].replace('\\','/') for x in list(test_loader.dataset.imgs)])
results['label'] = pd.Series(predictions)

In [48]:
results['label'] = results['label'].apply(lambda x: x.split('.')[1].title())
results['label']

0           Watch-101
1           Watch-101
2       American-Flag
3                Toad
4                Toad
            ...      
9172             Toad
9173             Toad
9174             Toad
9175             Toad
9176             Toad
Name: label, Length: 9177, dtype: object

In [49]:
results.to_csv("Group11_Kaggle2_Submission_v307_2307.csv")

In [ ]:
results.head(15)

In [ ]:
# results.head(5)

In [ ]:
from google.colab import files
# files.download('Group4_Pred_Submission_v75_lC_withques.csv')
files.download('Group11_Kaggle2_Submission_v307_2107.csv')
files.download('/content/CalTech-256-GoogleNet_v307_2107.pth')

In [ ]:
# submit the file to kaggle
# !kaggle competitions submit classification-of-caltech-256-images -f Group11_Kaggle2_Submission_v4_adv.csv -m "Model"

### Report Analysis

- Compare the accuracies for the Pre-trained vs CNN models
- What process was followed to tune the hyperparameters?
- Plot the confusion matrix in terms of the misclassifications